In [1]:
import os
import importlib.util
import torch.distributed as dist
import torch
from data.builder import build_dataset
from models.detectors.zid_rcnn import ZidRCNN
from scripts import dist_util
from models.utils.data_container import collate
from functools import partial
from PIL import Image
import numpy as np
from matplotlib import pyplot as plt
from tqdm import tqdm

In [2]:
def build_detector(model_cfg):
    model_cfg_ = model_cfg.copy()

    model_type = model_cfg_.pop('type') 
    assert model_type == 'ZidRCNN', f'{model_type} is not implemented yet.'
    return ZidRCNN(**model_cfg_)
    
def get_config_from_file(filename, mode):
    spec = importlib.util.spec_from_file_location(mode, filename)
    module = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(module)

    # Create a dictionary from module attributes
    config_dict = {key: getattr(module, key) for key in dir(module) if not key.startswith('__')}
    return config_dict

In [3]:
def test_batch_processing(batch):
    batch['obj_id'] = batch['id'][0]
    batch.pop('id')
    for k, v in batch.items():
        if k == 'img_metas':
            batch[k] = batch[k][0].data
        if k == 'img':
            batch[k] = [batch[k][0].data[0].to(device, non_blocking=True)]
        
        elif k in ['rgb', 'mask', 'traj']:
            batch[k] = batch[k].data.to(device, non_blocking=True)
    return batch

In [4]:
cfg = get_config_from_file('configs/test_conf.py', 'detection')
cfg.get('model')['train_cfg'] = None

In [5]:
dataset = build_dataset(cfg.get('data')['test'])

loading annotations into memory...
Done (t=0.13s)
creating index...
index created!


In [6]:
model = build_detector(cfg.get('model'))
# shared_drive_cv/FewshotObjectDetection/outputs/VoxDet_p1/iter_19201.pth
model_path = '/home/minhnh/project_drive/CV/FewshotObjectDetection/outputs/VoxDet_p2_1/iter_56251.pth'
model.load_state_dict(
    torch.load(model_path, map_location="cpu")['state_dict']
)
model.CLASSES = dataset.CLASSES
device = torch.device('cuda:4')
model.to(device)
model.eval()
print('Loaded model')

load model from: torchvision://resnet50
LOAD Backbone ResNET50 torchvision://resnet50


RuntimeError: Error(s) in loading state_dict for ZidRCNN:
	Missing key(s) in state_dict: "roi_head.roi_feat_extract.extract.1.weight", "roi_head.roi_feat_extract.extract.1.bias", "roi_head.roi_feat_extract.extract.2.weight", "roi_head.roi_feat_extract.extract.2.bias", "roi_head.roi_feat_extract.extract.2.running_mean", "roi_head.roi_feat_extract.extract.2.running_var", "roi_head.roi_feat_extract.extract.5.weight", "roi_head.roi_feat_extract.extract.5.bias", "roi_head.roi_feat_extract.extract.6.weight", "roi_head.roi_feat_extract.extract.6.bias", "roi_head.roi_feat_extract.extract.6.running_mean", "roi_head.roi_feat_extract.extract.6.running_var", "roi_head.roi_feat_extract.extract.9.weight", "roi_head.roi_feat_extract.extract.9.bias". 

In [7]:
data = torch.utils.data.DataLoader(dataset,
                                    batch_size=1,
                                    num_workers=2,
                                    shuffle=False,
                                    pin_memory=False,
                                    collate_fn=partial(collate, samples_per_gpu=1))

In [ ]:
results = []
torch.multiprocessing.set_sharing_strategy('file_system')
for batch in tqdm(data):
    batch = test_batch_processing(batch)
    with torch.no_grad():
        output = model(**batch, return_loss=False, rescale=True)
    results.extend(output)
    # break

  0%|          | 0/9109 [00:00<?, ?it/s]

Cls_scores 5
NMS Pre 2000
Cls_scores torch.Size([1, 192, 336])
bbox_preds torch.Size([4, 192, 336])
Anchr torch.Size([64512, 4])
Cls_scores torch.Size([2000])
Cls_scores torch.Size([1, 96, 168])
bbox_preds torch.Size([4, 96, 168])
Anchr torch.Size([16128, 4])
Cls_scores torch.Size([2000])
Cls_scores torch.Size([1, 48, 84])
bbox_preds torch.Size([4, 48, 84])
Anchr torch.Size([4032, 4])
Cls_scores torch.Size([2000])
Cls_scores torch.Size([1, 24, 42])
bbox_preds torch.Size([4, 24, 42])
Anchr torch.Size([1008, 4])
Cls_scores torch.Size([1008])
Cls_scores torch.Size([1, 12, 21])
bbox_preds torch.Size([4, 12, 21])
Anchr torch.Size([252, 4])
Cls_scores torch.Size([252])
Valid anchors,  torch.Size([7260, 4])


/home/minhnh/python_venv/cv/lib/python3.9/site-packages/torch/nn/functional.py:4358: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  warnings.warn(
/home/minhnh/python_venv/cv/lib/python3.9/site-packages/torch/nn/functional.py:4296: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  warnings.warn(
  0%|          | 3/9109 [00:01<1:14:13,  2.04it/s]

Cls_scores 5
NMS Pre 2000
Cls_scores torch.Size([1, 192, 336])
bbox_preds torch.Size([4, 192, 336])
Anchr torch.Size([64512, 4])
Cls_scores torch.Size([2000])
Cls_scores torch.Size([1, 96, 168])
bbox_preds torch.Size([4, 96, 168])
Anchr torch.Size([16128, 4])
Cls_scores torch.Size([2000])
Cls_scores torch.Size([1, 48, 84])
bbox_preds torch.Size([4, 48, 84])
Anchr torch.Size([4032, 4])
Cls_scores torch.Size([2000])
Cls_scores torch.Size([1, 24, 42])
bbox_preds torch.Size([4, 24, 42])
Anchr torch.Size([1008, 4])
Cls_scores torch.Size([1008])
Cls_scores torch.Size([1, 12, 21])
bbox_preds torch.Size([4, 12, 21])
Anchr torch.Size([252, 4])
Cls_scores torch.Size([252])
Valid anchors,  torch.Size([7260, 4])
Cls_scores 5
NMS Pre 2000
Cls_scores torch.Size([1, 192, 336])
bbox_preds torch.Size([4, 192, 336])
Anchr torch.Size([64512, 4])
Cls_scores torch.Size([2000])
Cls_scores torch.Size([1, 96, 168])
bbox_preds torch.Size([4, 96, 168])
Anchr torch.Size([16128, 4])
Cls_scores torch.Size([2000])


  0%|          | 7/9109 [00:02<28:09,  5.39it/s]  

...

  4%|▍         | 357/9109 [00:34<12:38, 11.54it/s]

Cls_scores 5
NMS Pre 2000
Cls_scores torch.Size([1, 192, 336])
bbox_preds torch.Size([4, 192, 336])
Anchr torch.Size([64512, 4])
Cls_scores torch.Size([2000])
Cls_scores torch.Size([1, 96, 168])
bbox_preds torch.Size([4, 96, 168])
Anchr torch.Size([16128, 4])
Cls_scores torch.Size([2000])
Cls_scores torch.Size([1, 48, 84])
bbox_preds torch.Size([4, 48, 84])
Anchr torch.Size([4032, 4])
Cls_scores torch.Size([2000])
Cls_scores torch.Size([1, 24, 42])
bbox_preds torch.Size([4, 24, 42])
Anchr torch.Size([1008, 4])
Cls_scores torch.Size([1008])
Cls_scores torch.Size([1, 12, 21])
bbox_preds torch.Size([4, 12, 21])
Anchr torch.Size([252, 4])
Cls_scores torch.Size([252])
Valid anchors,  torch.Size([7260, 4])
Cls_scores 5
NMS Pre 2000
Cls_scores torch.Size([1, 192, 336])
bbox_preds torch.Size([4, 192, 336])
Anchr torch.Size([64512, 4])
Cls_scores torch.Size([2000])
Cls_scores torch.Size([1, 96, 168])
bbox_preds torch.Size([4, 96, 168])
Anchr torch.Size([16128, 4])
Cls_scores torch.Size([2000])


  4%|▍         | 359/9109 [00:34<12:38, 11.54it/s]

Cls_scores 5
NMS Pre 2000
Cls_scores torch.Size([1, 192, 336])
bbox_preds torch.Size([4, 192, 336])
Anchr torch.Size([64512, 4])
Cls_scores torch.Size([2000])
Cls_scores torch.Size([1, 96, 168])
bbox_preds torch.Size([4, 96, 168])
Anchr torch.Size([16128, 4])
Cls_scores torch.Size([2000])
Cls_scores torch.Size([1, 48, 84])
bbox_preds torch.Size([4, 48, 84])
Anchr torch.Size([4032, 4])
Cls_scores torch.Size([2000])
Cls_scores torch.Size([1, 24, 42])
bbox_preds torch.Size([4, 24, 42])
Anchr torch.Size([1008, 4])
Cls_scores torch.Size([1008])
Cls_scores torch.Size([1, 12, 21])
bbox_preds torch.Size([4, 12, 21])
Anchr torch.Size([252, 4])
Cls_scores torch.Size([252])
Valid anchors,  torch.Size([7260, 4])


  4%|▍         | 361/9109 [00:34<12:25, 11.73it/s]

In [ ]:
dataset.evaluate(results, jsonfile_prefix='results')

In [ ]:
python3.8 /home/minhnh/project_drive/CV/FewshotObjectDetection/VoxDet-simplified/train.py --cuda_devices 0 --phase detection --config /home/minhnh/project_drive/CV/FewshotObjectDetection/VoxDet-simplified/configs/train_detection_conf.py